In [ ]:
# numpy downgrade가 필요할 수 있음
# !python3 -m pip install numpy==1.20.0

In [7]:
# !pip install ensemble_boxes


In [1]:
import pandas as pd
from pycocotools.coco import COCO
import numpy as np
from ensemble_boxes import *



In [2]:
# ensemble csv files
submission_files = ['submission_latest_3.csv', 'submission_latest_4.csv'] # submission csv 파일 이름을 넣어주세요

submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
annotation = '../../../dataset/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [5]:
prediction_strings = []
file_names = []

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        # boxes, scores, labels = nms(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = soft_nms(box_list, scores_list, labels_list, iou_thr=iou_thr)
        # boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list,iou_thr=iou_thr)
        # boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,iou_thr=0.5,conf_type='box_and_model_avg')
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list,iou_thr=0.55)

        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.54356572265625, 1.0, 0.643754365234375, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.39150673828125, 1.0, 0.49202123046875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.539386748046875, 1.0, 0.64904994140625, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:66: UserWarning: Zero area box skipped: [0.5391869140625, 1.0, 0.64673154296875, 1.0].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/opt/conda/envs/detection/lib/p

In [6]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,7 0.7492848634719849 215.9990692138672 51.8916...,test/0000.jpg
1,5 0.6051363945007324 131.41134643554688 0.2908...,test/0001.jpg
2,4 0.5629739165306091 866.147216796875 244.2601...,test/0002.jpg
3,9 0.3633488714694977 156.30117797851562 275.67...,test/0003.jpg
4,1 0.402248740196228 198.8267059326172 255.8225...,test/0004.jpg
